In [1]:
from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from numpy import linalg as LA
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from keras_vggface.vggface import VGGFace
from keras.engine import Model

Using TensorFlow backend.


In [11]:
from imgaug import augmenters as iaa

In [2]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [3]:
img_list = []
y_list = []
ref = {}
i = 0
for file in os.listdir("Cropped/"):
    yi = int(file.split('_')[0]) - 1 
#     image = cv2.imread("Cropped/"+ file)
# #     image = cv2.imread("Cropped/"+ file)
#     image = cv2.resize(image,(224,224))
    img = preprocess_image("Cropped/"+ file)[0,:]
    img_list.append(img)
    ref[i] = file.split('_')[0]
    y_list.append(yi)
    i+=1
X = np.array(img_list)
y = np.array(y_list)

In [5]:
num_classes = 195
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [6]:
batch_size = 64

epochs = 1

In [3]:
# input image dimensions
# img_rows, img_cols = 28, 28

In [4]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.reshape(-1,224,224,3)/255
# x_test = x_test.reshape(-1,1,28,28)/255

In [7]:
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (585, 224, 224, 3)
585 train samples
195 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [8]:
# model construction
model = Sequential()
model.add(Convolution2D(
    input_shape=( 224, 224, 3), # height width
    filters=32,
    kernel_size=5,
    strides=1,
    padding='same',      # Padding method
))
model.add(Activation('relu'))
model.add(MaxPooling2D(
	pool_size=2,
	strides=2,

))
model.add(Convolution2D(
	64,5,strides=1,padding='same',data_format='channels_first'
))
model.add(Activation('relu'))
model.add(MaxPooling2D(
	2,2,'same'
))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [10]:
adam = optimizers.Adam(lr=1e-4)
model.compile(adam,'categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train[:1000],y_train[:1000],batch_size=batch_size,epochs=epochs)

Epoch 1/1
585/585 [==============================] - 121s 207ms/step - loss: 16.0992 - acc: 0.0000e+00


In [9]:
loss, accuracy = model.evaluate(x_test[:100],y_test[:100])
print('test loss :', loss)
print('test acc:', accuracy)

100/100 [==============================] - 0s 4ms/step
test loss : 1.917460699081421
test acc: 0.62
